# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole
from dotenv import load_dotenv

In [3]:
load_dotenv("azure.env")

True

In [4]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

In [5]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 09-Jun-2025 15:51:33


## 1. Grounding Bing Search

In [6]:
name = "Web search agent"
model = "gpt-4o-mini"

instructions = "You are an AI Agent that can do some web search using Bing"


### Project client

In [7]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [8]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [9]:
# Create the agent with the Bing grounding tool
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-agent",
        instructions="You are a helpful agent",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing
bing.definitions


Created agent, ID: asst_ZfTszQQr43arZfC933PKhDHZ


[{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbinggrounding002', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

In [10]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_BuI2JzYXTFpvsICthIrmWQOI


In [11]:
query = "When is the next NHL game for the finals?"

In [12]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_CylxnO81mmbuKzt4QsyRGwSR


In [13]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [14]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_kb2rVGuEboMDLPw0yT4GYbZG status: completed
Step step_E0k5fR7a3EGzvN22hxV4Ww5e status: completed
  Tool calls:
    Tool Call ID: call_hVOiBZY2WmBgPnZzwvnd5gjB
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=next NHL game finals schedule 2025



{'prompt_tokens': 1449, 'completion_tokens': 90, 'total_tokens': 1539, 'prompt_token_details': {'cached_tokens': 0}}

In [15]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The next NHL Stanley Cup Final game is Game 3, scheduled for Saturday, June 8, 2025. The series is between the Edmonton Oilers and the Florida Panthers【3:0†2025 Stanley Cup Final schedule - NHL.com】【3:4†2025 Stanley Cup Final Schedule and Results - NHL Rumors】.
URL Citation: [2025 Stanley Cup Final schedule - NHL.com](https://www.nhl.com/news/2025-stanley-cup-final-schedule-television-results)
URL Citation: [2025 Stanley Cup Final Schedule and Results - NHL Rumors](https://nhlrumors.com/2025-stanley-cup-final-schedule-and-results/2025/06/07/)


In [32]:
query = "find out the most popular programming language for 2025?"

In [33]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_meFZiqEBd03QPUi9xlwpwxhZ


In [34]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [35]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_WTgJlj6CIy27SuAimQAgEwsP status: completed
Step step_WS6mNyqamtu6SwSXAe3ZJgh1 status: completed
  Tool calls:
    Tool Call ID: call_rIdUmukii2uLnDm4BHjz1meM
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=most popular programming language 2025



{'prompt_tokens': 2841, 'completion_tokens': 117, 'total_tokens': 2958, 'prompt_token_details': {'cached_tokens': 0}}

In [28]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [29]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_XsWlcN2lNa5GWekPo5dAifAC


In [30]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [31]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_TVR3SlrhlBazlENsrdHzW3gS status: completed
Step step_iZtbfzaV52QQDzJGSwSVJveL status: completed
  Tool calls:
    Tool Call ID: call_UZ66WTz0FXs2OaQIRgu4qjwv
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=current weather Mexico City



{'prompt_tokens': 2248, 'completion_tokens': 199, 'total_tokens': 2447, 'prompt_token_details': {'cached_tokens': 0}}

### Post processing